In [ ]:
from dotenv import load_dotenv
from jira import JIRA

In [ ]:
load_dotenv()
domain = os.getenv('JIRA_DOMAIN')
username = os.getenv('JIRA_USERNAME')
api_token = os.getenv('JIRA_API_TOKEN')
project_key = os.getenv('JIRA_PROJECT_KEY')
jira_options = {'server': jira_url}
jira = JIRA(options=jira_options, basic_auth=(username, api_token))

# project id van Datateam MOSS+
project_key = 'DSO'

In [ ]:
# get all members in project with project_key
project = jira.project(project_key)

In [ ]:
[user for user in jira.search_assignable_users_for_projects('', project_key) if user.active]

In [ ]:
fields = jira.fields()

# Find the story points field
for f in fields:
    if 'story points' in f['name'].lower():  # Adjust the search term if necessary
        print("Field Name:", f['name'])
        print("Field ID:", f['id'])
        break

In [ ]:
f

In [ ]:
def get_team_members(project):
    ''' Get all team members in a project and return a dictionary with key = name and value = accountId 
    '''
    team_members = set()
    
    # Fetch issues in the project and add assignees to the team_members set
    for issue in jira.search_issues(f'project={project.key}', maxResults=1000):
        if issue.fields.assignee:
            team_members.add(issue.fields.assignee)

    # get all team members in dictionary with key = name and value = accountId, also get the role of the team member
    team_members_dict = {}
    for team_member in team_members:
        team_members_dict[team_member.displayName.split(' ')[0]] = team_member.accountId    

    return team_members_dict

In [ ]:
team = get_team_members(project)
team

In [ ]:
assigned_roles = {
    'business_analist': 'Fried',
    'informatie_analist': 'Fried',
    'data_engineer': 'Fried',
    'bi_specialist': 'Fried',
}

project_input = {
    'name': 'Test Dashboard',
    'sub_domain': 'S&B', # kies 1 van ['MV', 'ON', 'SUB', 'S&B']
    'sub_project': 'TD1',
    'description': 'Dit is een test dashboard.',
    'year': 2024,
    'quarter': 1,
    'team_members': assigned_roles
}

# assert that all names are in the team keys
for role in assigned_roles.values():
    assert role in team.keys()

In [ ]:
epic_base_title = f"[Epic {project_input['sub_domain']}_{project_input['sub_project']}_{project_input['year']}]"
feature_base_title = f"[Feature {project_input['sub_domain']}_{project_input['sub_project']}_{project_input['year']}]"
story_base_title = f"[{project_input['sub_domain']}_{project_input['sub_project']}_{project_input['year']}]"

In [ ]:
all_issues = []

epic_dict = {
    'issuetype': {'name': 'Epic'},
    'summary': f"{epic_base_title} {project_input['name']}",
    'description': project_input['description'],
    'project': {'key': project_key},
    'labels': [project_input['sub_domain'], project_input['sub_project']],
    'assignee': {'accountId': team['Fried']},
}

epic_issue = jira.create_issue(fields=epic_dict)
all_issues.append(epic_issue)
print(f"Epic Created: https://{domain}.atlassian.net/browse/{epic_issue.key}")

In [ ]:
# test feature
feature_dict = {
    'issuetype': {'name': 'Feature'},
    'summary': f"{feature_base_title} {project_input['name']}",
    'description': project_input['description'],
    'project': {'key': project_key},
    'labels': [project_input['sub_domain'], project_input['sub_project']],
    'assignee': {'accountId': team['Fried']},
    'parent': {'key': epic_issue.key},
}

feature_issue = jira.create_issue(fields=feature_dict)
all_issues.append(feature_issue)
print(f"Feature Created: https://{domain}.atlassian.net/browse/{feature_issue.key}")

In [ ]:
# test story
story_dict = {
    'issuetype': {'name': 'Story'},
    'summary': f"{story_base_title} {project_input['name']}",
    'description': project_input['description'],
    'project': {'key': project_key},
    'labels': [project_input['sub_domain'], project_input['sub_project']],
    'assignee': {'accountId': team['Fried']},
    'parent': {'key': epic_issue.key},
}

update_dict = {
    'customfield_10004': 3,
}

story_issue = jira.create_issue(fields=story_dict)
jira.create_issue_link('Relates', story_issue.key, feature_issue.key)
all_issues.append(story_issue)
print(f"Story Created: https://{domain}.atlassian.net/browse/{story_issue.key}")

In [ ]:
# add story points field to create issue screen
jira.add_field_to_create_meta(story_issue.key, 'customfield_10004')

In [ ]:
# delete all issues
for issue in all_issues:
    issue.delete()

In [ ]:
try:
    issue.update(update={'customfield_10004': 2.0})
except Exception as e:
    print(e)